---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [30]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [6]:
df = pd.DataFrame(doc, columns=['text'])
df


,text
0,03/25/93 Total time of visit (in minutes):\n
1,6/18/85 Primary Care Doctor:\n
2,sshe plans to move as of 7/8/71 In-Home Servic...
3,7 on 9/27/75 Audit C Score Current:\n
4,2/6/96 sleep studyPain Treatment Pain Level (N...
...,...
495,1979 Family Psych History: Family History of S...
496,therapist and friend died in ~2006 Parental/Ca...
497,2008 partial thyroidectomy\n
498,sPt describes a history of sexual abuse as a c...


In [7]:
# find the number of characters for each string in df['text']
df['text'].str.len()

0       43
1       29
2       55
3       36
4       63
      ... 
495     69
496     67
497     27
498    286
499    364
Name: text, Length: 500, dtype: int64

In [8]:
# find the number of tokens for each string in df['text']
df['text'].str.split().str.len()

0       7
1       4
2      10
3       7
4       9
       ..
495    10
496     8
497     3
498    46
499    64
Name: text, Length: 500, dtype: int64

In [9]:
# find how many times a digit occurs in each string
df['text'].str.count(r'\d')

0      6
1      5
2      4
3      6
4      5
      ..
495    4
496    4
497    4
498    4
499    4
Name: text, Length: 500, dtype: int64

In [10]:
# find all occurances of the digits
df['text'].str.findall(r'\d')

0      [0, 3, 2, 5, 9, 3]
1         [6, 1, 8, 8, 5]
2            [7, 8, 7, 1]
3      [7, 9, 2, 7, 7, 5]
4         [2, 6, 9, 6, 7]
              ...        
495          [1, 9, 7, 9]
496          [2, 0, 0, 6]
497          [2, 0, 0, 8]
498          [2, 0, 0, 5]
499          [1, 9, 8, 0]
Name: text, Length: 500, dtype: object

In [11]:
# group and find the mm dd and yyyy 
df['text'].str.findall(r'(\d{1,2}[/-]\d{1,2}[/-]\d{2,4})')

0      [03/25/93]
1       [6/18/85]
2        [7/8/71]
3       [9/27/75]
4        [2/6/96]
          ...    
495            []
496            []
497            []
498            []
499            []
Name: text, Length: 500, dtype: object

In [12]:
df['text'].str.findall(r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[\S]*[+\s]\d{1,2}[,]{0,1}[+\s]\d{4})')

0      []
1      []
2      []
3      []
4      []
       ..
495    []
496    []
497    []
498    []
499    []
Name: text, Length: 500, dtype: object

In [15]:
df['text'].str.findall(r'(\d{1,2}[+\s](?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[\S]*[+\s]\d{4})')

0      []
1      []
2      []
3      []
4      []
       ..
495    []
496    []
497    []
498    []
499    []
Name: text, Length: 500, dtype: object

In [16]:
df['text'].str.findall(r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[\S]*[+\s]\d{4})'
 )

0      []
1      []
2      []
3      []
4      []
       ..
495    []
496    []
497    []
498    []
499    []
Name: text, Length: 500, dtype: object

In [17]:
df['text'].str.findall(r'(\d{1,2}[/-][1|2]\d{3})')

0      []
1      []
2      []
3      []
4      []
       ..
495    []
496    []
497    []
498    []
499    []
Name: text, Length: 500, dtype: object

In [18]:
df['text'].str.findall(r'([1|2]\d{3})')

0          []
1          []
2          []
3          []
4          []
        ...  
495    [1979]
496    [2006]
497    [2008]
498    [2005]
499    [1980]
Name: text, Length: 500, dtype: object

In [19]:
parsed=df['text'].str.extract('(\d{1,2}[/-]\d{1,2}[/-]\d{2,4})')

In [20]:
parsed = parsed.iloc[:,0].str.replace('Janaury', 'January').str.replace('Decemeber', 'December')
    

In [21]:
parsed = pd.Series(pd.to_datetime(parsed))
 

In [22]:
parsed = parsed.sort_values(ascending=True).index
    

In [27]:
values = pd.Series(parsed.values)

In [28]:
values.head()

0     9
1    84
2     2
3    53
4    28
dtype: int64

In [31]:
    
def date_sorter():
    
    exp1 = '(\d{1,2}[/-]\d{1,2}[/-]\d{2,4})'
    exp2 = '((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[\S]*[+\s]\d{1,2}[,]{0,1}[+\s]\d{4})'
    exp3 = '(\d{1,2}[+\s](?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[\S]*[+\s]\d{4})'
    exp4 = '((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[\S]*[+\s]\d{4})'
    exp5 = '(\d{1,2}[/-][1|2]\d{3})'
    exp6 = '([1|2]\d{3})'
    expAll = '(%s|%s|%s|%s|%s|%s)' %(exp1, exp2, exp3, exp4, exp5, exp6)
    parsed = df.str.extract(expAll)
    parsed = pd.Series(pd.to_datetime(parsed))
    parsed = parsed.sort_values(ascending=True).index
    result = pd.Series(parsed.values)
    
    return result
date_sorter()

#    exp3 = '(\d{1,2}[+\s](?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[\S]*[+\s]\d{4})'
#    exp4 = '((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[\S]*[+\s]\d{4})'

AttributeError: 'int' object has no attribute 'lower'